## Imports

In [1]:
from __future__ import division, print_function, absolute_import
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
from math import ceil

## Helper functions
Helper functions borrowed from original paper by Li et al. 

In [2]:
def makedirs(path):
    '''
    if path does not exist in the file system, create it
    '''
    if not os.path.exists(path):
        os.makedirs(path)

def list_of_distances(X, Y):
    '''
    Given a list of vectors, X = [x_1, ..., x_n], and another list of vectors,
    Y = [y_1, ... , y_m], we return a list of vectors
            [[d(x_1, y_1), d(x_1, y_2), ... , d(x_1, y_m)],
             ...
             [d(x_n, y_1), d(x_n, y_2), ... , d(x_n, y_m)]],
    where the distance metric used is the sqared euclidean distance.
    The computation is achieved through a clever use of broadcasting.
    '''
    XX = torch.reshape(list_of_norms(X), shape=(-1, 1))
    YY = torch.reshape(list_of_norms(Y), shape=(1, -1))
    output = XX + YY - 2 * torch.mm(X, Y.t())

    return output

def list_of_norms(X):
    '''
    X is a list of vectors X = [x_1, ..., x_n], we return
        [d(x_1, x_1), d(x_2, x_2), ... , d(x_n, x_n)], where the distance
    function is the squared euclidean distance.
    '''
    return torch.sum(torch.pow(X, 2), dim=1)

def print_and_write(str, file):
    '''
    print str to the console and also write it to file
    '''
    print(str)
    file.write(str + '\n')

## Create necessary folders

In [3]:
# data folder
makedirs('./data/mnist')

# Models folder
model_folder = os.path.join(os.getcwd(), "saved_model", "mnist_model", "mnist_cae_1")
makedirs(model_folder)

# Image folder
img_folder = os.path.join(model_folder, "img")
makedirs(img_folder)

# Model filename
model_filename = "mnist_cae"

## Dataset - Pytorch
#### <font color='red'>Double check the normalization mean and stdev for dataset</font>
#### <font color='red'>Double check parameters Dataloader (e.g. shuffle on or off, different batch sizes for train/valid/test)</font>

In [4]:
# Transforms to perform on loaded dataset. Normalize around mean 0.1307 and std 0.3081 for optimal pytorch results. 
# source: https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457/4
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.1307,),(0.3081,))])

# Load datasets into reproduction/data/mnist. Download if data not present. 
mnist_train = DataLoader(torchvision.datasets.MNIST('./data/mnist', train=True, download=True, transform=transforms))

mnist_train_data = mnist_train.dataset.data
mnist_train_targets = mnist_train.dataset.targets

# first 55000 examples for training
x_train = mnist_train_data[0:55000]
y_train = mnist_train_targets[0:55000]

# 5000 examples for validation set
x_valid = mnist_train_data[55000:60000]
y_valid = mnist_train_targets[55000:60000]

# 10000 examples in test set

mnist_test = DataLoader(torchvision.datasets.MNIST('./data/mnist', train=False, download=True, 
                                                   transform=transforms))

x_test = mnist_test.dataset.data
y_test = mnist_test.dataset.targets

train_data = TensorDataset(x_train, y_train)
valid_data = TensorDataset(x_valid, y_valid)
test_data = TensorDataset(x_test, y_test)

batch_size = 250

# Datasets in DataLoader, can be used in iteration: for (x, y) in train_dl...
# Check parameters
train_dl = DataLoader(train_data, batch_size=batch_size, drop_last=False, shuffle=False)
valid_dl = DataLoader(valid_data, batch_size=batch_size, drop_last=False, shuffle=False)
test_dl = DataLoader(test_data, batch_size=batch_size, drop_last=False, shuffle=False)

100.1%

Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz


28.4%

113.5%

Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz


100.4%

Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz


180.4%

Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


## Parameters

In [5]:
# COPIED FROM THE ORIGINAL IMPLEMENTATION
# training parameters
learning_rate = 0.002
training_epochs = 1500

# frequency of testing and saving
test_display_step = 100   # how many epochs we do evaluate on the test set once
save_step = 50            # how frequently do we save the model to disk

# elastic deformation parameters
sigma = 4
alpha = 20

# lambda's are the ratios between the four error terms
lambda_class = 20
lambda_ae = 1 # autoencoder
lambda_1 = 1 # push prototype vectors to have meaningful decodings in pixel space
lambda_2 = 1 # cluster training examples around prototypes in latent space


input_height = input_width =  28    # MNIST data input shape 
n_input_channel = 1     # the number of color channels; for MNIST is 1.
input_size = input_height * input_width * n_input_channel   # 784
n_classes = 10

# Network Parameters
n_prototypes = 15         # the number of prototypes
n_layers = 4

# height and width of each layers' filters
f_1 = 3
f_2 = 3
f_3 = 3
f_4 = 3

# stride size in each direction for each of the layers
s_1 = 2
s_2 = 2
s_3 = 2
s_4 = 2

# number of feature maps in each layer
n_map_1 = 32
n_map_2 = 32
n_map_3 = 32
n_map_4 = 10

# the shapes of each layer's filter
# [out channel, in_channel, 3, 3]
filter_shape_1 = [n_map_1, n_input_channel, f_1, f_1]
filter_shape_2 = [n_map_2, n_map_1, f_2, f_2]
filter_shape_3 = [n_map_3, n_map_2, f_3, f_3]
filter_shape_4 = [n_map_4, n_map_3, f_4, f_4]

# strides for each layer (changed to tuples)
stride_1 = [s_1, s_1]
stride_2 = [s_2, s_2]
stride_3 = [s_3, s_3]
stride_4 = [s_4, s_4]


## Initialize encoder and decoder

## Model construction

In [6]:
# Dummy tensor for debugging
# X = torch.empty(batch_size, n_input_channel, input_width, input_height)
X,Y = next(iter(train_dl))
X = X.view(250,1,28,28).float()
# X in shape 250x28x28 (Batch x H x W)
print(X.shape)
# Y in shape 250 (B). Needs to be converted to one-hot for reproduction of paper

torch.Size([250, 1, 28, 28])


## Model
#### <font color='red'>Fix the stride and padding parameters, check if filter in tf is same as weight in pt</font>
Padding discussion pytorch: https://github.com/pytorch/pytorch/issues/3867

Blogpost: https://mmuratarat.github.io/2019-01-17/implementing-padding-schemes-of-tensorflow-in-python

In [7]:
class Encoder(nn.Module):
    '''Encoder'''
    def __init__(self):
        super(Encoder, self).__init__()
        
        # height and width of each layers' filters
        f_1 = 3
        f_2 = 3
        f_3 = 3
        f_4 = 3
        
        # define layers
        self.enc_l1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=0)
        self.enc_l2 = nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=0)
        self.enc_l3 = nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=0)
        self.enc_l4 = nn.Conv2d(32, 10, kernel_size=3, stride=2, padding=0)
        
    def pad_image(self, img):
        ''' Takes an input image (batch) and pads according to Tensorflows SAME padding'''
        input_h = img.shape[2]
        input_w = img.shape[3]
        stride = 2 
        filter_h = 3
        filter_w = 3

        output_h = int(ceil(float(input_h)) / float(stride))
        output_w = output_h

        if input_h % stride == 0:
            pad_height = max((filter_h - stride), 0)
        else:
            pad_height = max((filter_h - (input_h % stride), 0))

        pad_width = pad_height

        pad_top = pad_height // 2
        pad_bottom = pad_height - pad_top
        pad_left = pad_width // 2
        pad_right = pad_width - pad_left

        padded_img = torch.zeros(img.shape[0], img.shape[1], input_h + pad_height, input_w + pad_width)
        padded_img[:,:, pad_top:-pad_bottom, pad_left:-pad_right] = img

        return padded_img
        
    def forward(self, x):
        pad_x = self.pad_image(x)
        el1 = self.enc_l1(pad_x)
        
        pad_el1 = self.pad_image(el1)
        el2 = self.enc_l2(pad_el1)
    
        pad_el2 = self.pad_image(el2)
        el3 = self.enc_l3(pad_el2)
        
        pad_el3 = self.pad_image(el3)
        el4 = self.enc_l4(pad_el3)
        
        return el4
        
class Decoder(nn.Module):
    '''Decoder'''
    def __init__(self):
        super(Decoder, self).__init__()
        # height and width of each layers' filters
        f_1 = 3
        f_2 = 3
        f_3 = 3
        f_4 = 3

        # define layers
        self.dec_l4 = nn.ConvTranspose2d(10, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.dec_l3 = nn.ConvTranspose2d(32, 32, kernel_size=3, stride=2, padding=1, output_padding=0)
        self.dec_l2 = nn.ConvTranspose2d(32, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.dec_l1 = nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1)

    def forward(self, enc_x):
        dl4 = self.dec_l4(enc_x)
        dl3 = self.dec_l3(dl4)
        dl2 = self.dec_l2(dl3)
        decoded_x = self.dec_l1(dl2)
        
        return decoded_x
        
class nn_prototype(nn.Module):
    '''Model'''
    def __init__(self, n_prototypes=15, n_layers=4, n_classes=10):
        super().__init__()
        
        self.encoder = Encoder()
        self.decoder = Decoder()
        
        # initialize prototype - currently not in correct spot
        n_features = 40 # size of encoded x - 250 x 10 x 2 x 2
        self.prototype_feature_vectors = nn.Parameter(torch.empty(size=(n_prototypes, n_features), dtype=torch.float32).uniform_())
        
        self.last_layer = nn.Linear(n_prototypes,10)
        
    def list_of_distances(self, X, Y):
        '''
        Given a list of vectors, X = [x_1, ..., x_n], and another list of vectors,
        Y = [y_1, ... , y_m], we return a list of vectors
                [[d(x_1, y_1), d(x_1, y_2), ... , d(x_1, y_m)],
                 ...
                 [d(x_n, y_1), d(x_n, y_2), ... , d(x_n, y_m)]],
        where the distance metric used is the sqared euclidean distance.
        The computation is achieved through a clever use of broadcasting.
        '''
        XX = torch.reshape(list_of_norms(X), shape=(-1, 1))
        YY = torch.reshape(list_of_norms(Y), shape=(1, -1))
        output = XX + YY - 2 * torch.mm(X, Y.t())

        return output

    def list_of_norms(self, X):
        '''
        X is a list of vectors X = [x_1, ..., x_n], we return
            [d(x_1, x_1), d(x_2, x_2), ... , d(x_n, x_n)], where the distance
        function is the squared euclidean distance.
        '''
        return torch.sum(torch.pow(X, 2), dim=1)
    
    def forward(self, x):
        
        #encoder step
        enc_x = self.encoder(x)
        
        #decoder step
        dec_x = self.decoder(enc_x)
        # hardcoded input size
        dec_x = dec_x.view(x.shape[0], x.shape[1], x.shape[2], x.shape[3])
        
        # flatten encoded x to compute distance with prototypes
        n_features = enc_x.shape[1] * enc_x.shape[2] * enc_x.shape[3]
        feature_vectors_flat = torch.reshape(enc_x, shape=[-1, n_features])
        
        # distance to prototype
        prototype_distances = list_of_distances(feature_vectors_flat, self.prototype_feature_vectors)
        
        # distance to feature vectors
        feature_vector_distances = list_of_distances(self.prototype_feature_vectors, feature_vectors_flat)
        
        # classification layer
        logits = self.last_layer(prototype_distances)
        
        # Softmax to prob dist not needed as cross entropy loss is used?
        
        return dec_x, logits, feature_vector_distances, prototype_distances
        

## Cost function

In [8]:
'''
the error function consists of 4 terms, the autoencoder loss,
the classification loss, and the two requirements that every feature vector in
X look like at least one of the prototype feature vectors and every prototype
feature vector look like at least one of the feature vectors in X.
'''
def loss_function(X_decoded, X_true, logits, Y, feature_dist, prototype_dist, lambdas=None):
    if lambdas == None:
        lambda_class, lambda_ae, lambda_1, lambda_2 = 20, 1, 1, 1
    
    batch_size = X_true.shape[0]
    
    ae_error = torch.mean(list_of_norms(X_decoded - X_true))
    class_error = F.cross_entropy(logits, Y)
    error_1 = torch.mean(torch.min(feature_dist, axis=1)[0])
    error_2 = torch.mean(torch.min(prototype_dist, axis = 1)[0])

    # total_error is the our minimization objective
    total_error = lambda_class * class_error +\
                  lambda_ae * ae_error + \
                  lambda_1 * error_1 + \
                  lambda_2 * error_2

    return total_error/batch_size

## Accuracy
### <font color='red'> Currently not working properly </font>

In [58]:
def compute_acc(logits, labels, batch_size):
    predictions = logits.argmax(dim=1)
    total_correct = torch.sum(predictions == labels).item()
    accuracy = total_correct / batch_size
    
    return(accuracy)

## Training loop

In [63]:
def train():
    model = nn_prototype(15,4,10)
#     print("Model parameters:")
#     for name, p in model.named_parameters():
#         print(name)
#     print("\n")
    
    # initialize optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # initialize storage for results
    train_acc = []
    train_losses = []
    test_acc = []
    test_losses = []
    valid_acc = []
    valid_losses = []

    # training loop
    for epoch in range(training_epochs):
        print("epoch: ", epoch)

        # loop over the batches
        for step, (x, Y) in enumerate(train_dl):
            # x is non_flattened batch of images (since using convnets)

            optimizer.zero_grad()
            x = x.view(x.shape[0], 1, x.shape[1], x.shape[2]).float()
            
            # perform forward pass
            X_decoded, logits, feature_dist, prot_dist = model(x)

            # compute the loss
            total_loss = loss_function(X_decoded, x, logits, Y, feature_dist, prot_dist)

            # backpropagate over the loss
            total_loss.backward()
            
            # update the weights
            optimizer.step()

            # compute and save accuracy and loss
            accuracy = compute_acc(F.softmax(logits, dim=1), Y, batch_size)
            train_acc.append(accuracy)
            train_losses.append(total_loss.item())
    
            # print information
            if step % 20 == 0:
                print('\nStep: ', step)
                print('Train loss: ', total_loss.item())
                print('Train acc: ', accuracy)
        
        # testing
        if epoch % test_display_step == 0:
            # perform testing
            with torch.no_grad():
                for step, (x_test, y_test) in enumerate(test_dl):
                    x_test = x.view(x_test.shape[0], 1, x_test.shape[1], x_test.shape[2]).float()
                    
                    # forward pass
                    X_decoded, logits, feature_dist, prot_dist = model(x_test)

                    # compute loss and accuracy and save
                    total_loss = loss_function(X_decoded, x_test, logits, y_test, feature_dist, prot_dist)
                    test_accuracy = compute_acc(logits, y_test, batch_size)
                    test_loss = loss_function(X_decoded, x_test, logits, y_test, feature_dist, prot_dist)
                    test_acc.append(test_accuracy)
                    test_losses.append(test_loss)
                    
                print('Test loss: ', test_loss.item())
                print('Test acc: ', test_accuracy)
                    
        # validation
        with torch.no_grad():
            for step, (x_valid, y_valid) in enumerate(valid_dl):
                    x_valid = x.view(x_valid.shape[0], 1, x_valid.shape[1], x_valid.shape[2]).float()
                    X_decoded, logits, feature_dist, prot_dist = model(x_valid)
                    total_loss = loss_function(X_decoded, x_valid, logits, y_valid, feature_dist, prot_dist)
                    valid_accuracy = compute_acc(logits, y_valid, batch_size)
                    valid_loss = loss_function(X_decoded, x_valid, logits, y_valid, feature_dist, prot_dist)
                    valid_acc.append(valid_accuracy)
                    valid_losses.append(valid_loss)
            print('Valid loss: ', valid_loss.item())
            print('Valid acc: ', valid_accuracy)
        
    return train_acc, train_loss, test_acc, test_loss
        

In [ ]:
train()

epoch:  0

Step:  0
Train loss:  169.23463439941406
Train acc:  0.092

Step:  20
Train loss:  32.5084114074707
Train acc:  0.088

Step:  40
Train loss:  21.734891891479492
Train acc:  0.156

Step:  60
Train loss:  17.311674118041992
Train acc:  0.128

Step:  80
Train loss:  17.28351402282715
Train acc:  0.172

Step:  100
Train loss:  15.273221969604492
Train acc:  0.38

Step:  120
Train loss:  14.073651313781738
Train acc:  0.388

Step:  140
Train loss:  12.967559814453125
Train acc:  0.312

Step:  160
Train loss:  11.939696311950684
Train acc:  0.48

Step:  180
Train loss:  11.770564079284668
Train acc:  0.464

Step:  200
Train loss:  10.481990814208984
Train acc:  0.456

Epoch 0 finished
Test loss:  10.159423828125
Test acc:  0.416
Test loss:  10.267666816711426
Test acc:  0.096
Valid loss:  10.264192581176758
Valid acc:  0.084
epoch:  1

Step:  0
Train loss:  9.999153137207031
Train acc:  0.5

Step:  20
Train loss:  9.716378211975098
Train acc:  0.54

Step:  40
Train loss:  8.785962